# Week 4: Transfer Learning, BERT (Homework)

## Question Search Engine

Embeddings are a good source of information for solving various tasks. For example, we can classify texts or find similar documents using their representations. We already know about word2vec, GloVe and fasttext, but they don't use context information from given text (only from contexts of source data).

For today we will use full power of context-aware embeddings to find text duplicates!

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [1]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 16.9 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.18.3-py3-none-any.whl size=1770192 sha256=f723d7f54fec1026f0fdf4c562eab07dd4401aec4dd66b6241d11b8219c14f8f
  Stored in directory: /root/.cache/pip/wheels/c9/9a/37/beb534d37a37cd057d48ba20b82f34d527816b7fdf0206882f
Successfully built deepspeed
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-

### Data Preparation

In [2]:
qqp = datasets.load_dataset("SetFit/qqp")
print("\n")
print("Sample[0]:", qqp["train"][0])
print("Sample[3]:", qqp["train"][3])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/70.8M [00:00<?, ?B/s]

validation.jsonl: 0.00B [00:00, ?B/s]

test.jsonl:   0%|          | 0.00/76.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [3]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [4]:
MAX_LENGTH = 128

def preprocess_function(examples):
    result = tokenizer(
        examples["text1"],
        examples["text2"],
        padding="max_length",
        max_length=MAX_LENGTH,
        truncation=True,
    )

    result["label"] = examples["label"]

    return result

In [5]:
qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [6]:
print(repr(qqp_preprocessed["train"][0]["input_ids"])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Evaluation (1 point)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [7]:
val_set = qqp_preprocessed["validation"]
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [8]:
for batch in val_loader:
    break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
    predicted = model(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        token_type_ids=batch["token_type_ids"],
    )

print("\nPrediction (probs):", torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

**Task 1 (1 point)**

- Measure the validation accuracy of your model. Doing so naively may take several hours. Please make sure you use the following optimizations:
  - Run the model on GPU with no_grad
  - Using batch size larger than 1
  - Use optimize data loader with num_workers > 1
  - (Optional) Use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=64,
    shuffle=False,
    num_workers=4,
    collate_fn=transformers.default_data_collator
)

correct = 0
total = 0

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            token_type_ids=batch["token_type_ids"],
        )

        preds = outputs.logits.argmax(dim=1)
        correct += (preds == batch["labels"]).sum().item()
        total += batch["labels"].size(0)

accuracy = correct / total
accuracy

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


0.9083848627256987

In [10]:
assert 0.9 < accuracy < 0.91

### Training (4 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

**Task 2 (4 points)**
- Choose Option A or Option B (only one will be graded)
- Follow all the instructions and restrictions

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

bert_base_name = "textattack/bert-base-uncased-QQP"
tokenizer2 = AutoTokenizer.from_pretrained(bert_base_name)
model2 = AutoModelForSequenceClassification.from_pretrained(bert_base_name)

roberta_name = "JeremiahZ/roberta-base-qqp"
tokenizer3 = AutoTokenizer.from_pretrained(roberta_name)
model3 = AutoModelForSequenceClassification.from_pretrained(roberta_name)

distillbert_name = "textattack/distilbert-base-uncased-QQP"
tokenizer4 = AutoTokenizer.from_pretrained(distillbert_name)
model4 = AutoModelForSequenceClassification.from_pretrained(distillbert_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/484 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [14]:
import time
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def evaluate_best(model, tokenizer, q1, q2, y_true, batch_size=64, max_length=MAX_LENGTH):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()
    torch.set_grad_enabled(False)
    model_size = sum(p.numel() for p in model.parameters()) * 4 / (1024 * 1024)

    all_preds = []
    all_labels = []
    start_time = time.time()

    for i in range(0, len(q1), batch_size):
        enc = tokenizer(
            q1[i:i+batch_size],
            q2[i:i+batch_size],
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt",
        )
        enc = {k: v.to(device) for k, v in enc.items()}

        outputs = model(**enc)
        preds = outputs.logits.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_true[i:i+batch_size])

    end_time = time.time()
    #speed
    inference_time = end_time - start_time
    total_samples = len(q1)
    samples_per_second = total_samples / inference_time
    #metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    torch.set_grad_enabled(True)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "samples_per_second": samples_per_second,
        "model_size_mb": model_size,
        "inference_time_seconds": inference_time
    }

In [15]:
import pandas as pd

q1, q2, y = qqp["validation"]["text1"], qqp["validation"]["text2"], qqp["validation"]["label"]

models = [
    ("bert-base-cased-finetuned-qqp", tokenizer, model),
    ("bert-base-uncased-QQP", tokenizer2, model2),
    ("roberta-base-qqp",      tokenizer3, model3),
    ("distilbert-base-uncased-QQP",   tokenizer4, model4),
]

rows = []
for name, tknz, mdl in models:
    m = evaluate_best(mdl, tknz, q1, q2, y)
    rows.append({"model": name, **{k: round(v, 4) for k, v in m.items()}})

pd.DataFrame(rows).sort_values("accuracy", ascending=False)

,model,accuracy,precision,recall,f1,samples_per_second,model_size_mb,inference_time_seconds
2,roberta-base-qqp,0.9153,0.8732,0.9008,0.8868,251.8500,475.4912,160.5321
1,bert-base-uncased-QQP,0.9091,0.8665,0.8902,0.8782,229.5757,417.6475,176.1075
0,bert-base-cased-finetuned-qqp,0.6318,0.0000,0.0000,0.0000,546.3802,703.5205,73.9961
3,distilbert-base-uncased-QQP,0.5339,0.4378,0.9362,0.5966,461.1043,255.4131,87.6808


### Finding Duplicates (1 point)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

**Task 3 (1 point)**
- Implement function for finding duplicates
- Test it on several examples (at least 5)
- Check suggested duplicates and make a conclusion about model correctness

In [23]:
def get_embeddings_fast(model, tokenizer, texts, batch_size=128):
    model.eval()
    device = next(model.parameters()).device
    all_embeddings = []

    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]

            enc = tokenizer(
                batch,
                padding=True,
                truncation=True,
                max_length=128,
                return_tensors="pt"
            )
            enc = {k: v.to(device) for k, v in enc.items()}

            outputs = model.base_model(**enc)
            cls_emb = outputs.last_hidden_state[:, 0]
            cls_emb = F.normalize(cls_emb, dim=1)

            all_embeddings.append(cls_emb)

    return torch.cat(all_embeddings, dim=0).cpu()

In [24]:
N = 50000
train_questions = qqp["train"]["text1"][:N]
train_embeddings = get_embeddings(
    model3,
    tokenizer3,
    train_questions,
    batch_size=64
)

In [25]:
def find_duplicates(question, model, tokenizer, train_questions, train_embeddings, top_k=5):
    model.eval()
    device = next(model.parameters()).device

    with torch.no_grad():
        enc = tokenizer(
            question,
            return_tensors="pt",
            truncation=True,
            max_length=128
        ).to(device)

        output = model.base_model(**enc)
        q_emb = output.last_hidden_state[:, 0]
        q_emb = F.normalize(q_emb, dim=1).cpu()

    similarities = torch.matmul(train_embeddings, q_emb.T).squeeze(1)
    top_idx = similarities.topk(top_k).indices

    return [(train_questions[i], similarities[i].item()) for i in top_idx]

In [26]:
test_questions = [
    "How can I lose weight fast?",
    "What is the capital of France?",
    "How to learn Python programming?",
    "What causes climate change?",
    "How do neural networks work?"
]

for q in test_questions:
    print("\nQUESTION:", q)
    results = find_duplicates(
        q,
        model3,
        tokenizer3,
        train_questions,
        train_embeddings
    )
    for text, score in results:
        print(f"  {score:.3f} | {text}")


QUESTION: How can I lose weight fast?
  1.000 | What are the properties of indifference curve?
  1.000 | What can I do to improve my question on Quora?
  1.000 | How do I participate or contribute in open source projects?
  1.000 | How can you prevent jet lag?
  1.000 | What if India bans import of all Chinese products?

QUESTION: What is the capital of France?
  1.000 | What do Pakistanis think of Indians?
  1.000 | Catholics: how do I stop watching internet porn?
  1.000 | Are you proud of yourself? Why?
  1.000 | Should I watch Game of Thrones before I read the books?
  1.000 | Should I watch Game of Thrones before I read the books?

QUESTION: How to learn Python programming?
  1.000 | How can I enter into bollywood?
  1.000 | What will be the implications of banning 500 and 1000 rupees currency notes on Indian economy?
  1.000 | What will be the implications of banning 500 and 1000 rupees currency notes on Indian economy?
  1.000 | What will be the implications of banning 500 and 

### Bonus: Finding Duplicates Faster (0.5 point)

Try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.

**Bonus Task 1 (0.5 point)**
- Speed up your implementation from "Finding Duplicates" part
- Capture both old and new implementation work time
- Describe your approach

In [ ]:
<A whole lot of YOUR CODE HERE>

### Bonus: Finding Duplicates in Old-Fashioned way (1.5 points)

In this bonus task you are supposed to use pretrained embeddings (word2vec, GloVe or fasttext) for solving the duplicates problem.

**Bonus Task 2 (1.5 points)**
- Solve Finding Duplicates problem using mentioned embeddings
- Compare old-fashioned solution to previous ones (quality, speed, etc.)
- Make a small report (up to 5 steps, results and conclusions) on work done in this part

In [ ]:
<A whole lot of YOUR CODE HERE>